In [6]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install huggingface_hub
!pip install .
%cd examples/dreambooth
!pip install -r requirements.txt
!pip install transformers

Cloning into 'diffusers'...
remote: Enumerating objects: 52265, done.
remote: Counting objects: 100% (11911/11911), done.
remote: Compressing objects: 100% (990/990), done.
remote: Total 52265 (delta 11584), reused 10956 (delta 10908), pack-reused 40354
Receiving objects: 100% (52265/52265), 35.17 MiB | 25.38 MiB/s, done.
Resolving deltas: 100% (37698/37698), done.
/workspace/diffusers
  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/70/25/fab23259a52ece5670dcb8452e1af34b89e6135ecc17cd4b54b4b479eac6/fsspec-2023.12.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 47.7 MB/s eta 0:00:00

[notice] A ne

In [12]:
from huggingface_hub import login
login()

In [15]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"  \
  --instance_data_dir="/workspace/train_image" \
  --output_dir="/workspace/saved_model" \
  --instance_prompt="a photo of a super hero in cartoon" \
  --resolution=512 \
  --train_text_encoder \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=300

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
02/03/2024 10:41:00 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'timestep_spacing', 'prediction_type', 'sample_max_value', 'clip_sample_range', 'rescale_betas_zero_snr',

In [16]:
from diffusers import DiffusionPipeline
import torch

In [19]:
pipeline = DiffusionPipeline.from_pretrained("/workspace/saved_model", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
image = pipeline("A photo of cartoon hero in a bucket", num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("dog-bucket.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]